In [1]:
import pickle
from pathlib import Path
import importlib.util
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.fft as fft
from scipy.signal import find_peaks, welch, get_window, periodogram
from scipy.optimize import curve_fit
import matplotlib

matplotlib.use("Qt5Agg")

path = Path("").absolute()
path_to_mod = str(path.parents[0] / "johannes_tools" / "data_analysis" / "fitting.py")
spec = importlib.util.spec_from_file_location("fitting", path_to_mod)
johannes_tools = importlib.util.module_from_spec(spec)
sys.modules["fitting"] = johannes_tools
spec.loader.exec_module(johannes_tools)
from fitting import evaluate_hyperfine, fit_hyperfine # noqa



# Load Data and fit ZSCs

In [2]:
#--- Load the ODMR data, evaluate hyperfine spectrum, including zero crossing slopes ---#

odmr_data = pd.read_csv("20240326-1823-44_85mW_hyperfine_ohne_sinc_ODMR_signal.dat", comment='#', sep='\t', header=None, names=['Frequency', 'Voltage'])

fig, ax = plt.subplots()

#--- Find peaks and dips in the ODMR signal; and fit slopes of zero-crossings ---#
peaks_indices, dips_indices, zero_crossings_indices, slopes, intercepts = evaluate_hyperfine(np.array(odmr_data['Frequency']), np.array(odmr_data['Voltage']), 
                                                                                             feature_distance_in_Hz=0.5e6, min_feature_amplitude=0.02, zero_crossings_fit_range=0.1e6)


print(odmr_data["Frequency"].iloc[peaks_indices])

ax.scatter(odmr_data["Frequency"], odmr_data["Voltage"], s=1)
ax.scatter(odmr_data['Frequency'].iloc[peaks_indices], odmr_data['Voltage'].iloc[peaks_indices], color='red')
ax.scatter(odmr_data['Frequency'].iloc[dips_indices], odmr_data['Voltage'].iloc[dips_indices], color='blue')
for i, slope in enumerate(slopes):
    ax.plot(odmr_data['Frequency'], slopes[i]*odmr_data['Frequency'] + intercepts[i], color='green')

ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Voltage [V]")
ax.set_title("Fitting ZCSs")
ax.grid()
fig.tight_layout()
fig.show()


# factors for converting voltage values to magnetic field values
gyromagnetic_ratio_Hz_per_nT = 28.024 # Hz/nT
slope_in_V_per_nT = slopes[1] * gyromagnetic_ratio_Hz_per_nT
volts_to_nT_multiplication_factor = 1/slopes[1] * 1/gyromagnetic_ratio_Hz_per_nT # 1/slope is Hz/V, 1/gyromagnetic ratio is nT/Hz


#--- Load the saved data from the lock-in amplifier ---#
with open('saved_result.pkl', 'rb') as f:
    result = pickle.load(f)


data_dict_x = result['/dev7279/demods/0/sample.x'][0]
data_dict_y = result['/dev7279/demods/0/sample.y'][0]

times = data_dict_x.time
duration = result['/duration'][0] # [s]
sample_rate = int(len(times)/duration) # [Hz]

# apply conversion factor to the data
samples_x_B_field = 100*data_dict_x.value[0] * volts_to_nT_multiplication_factor
samples_x_B_field_original = samples_x_B_field.copy()

# # remove the DC offset in each 1 second interval separately -> unsure if this is good practice
# for i in range(int(result['/duration'][0])):
#     samples_x_B_field[i*sample_rate:(i+1)*sample_rate] = samples_x_B_field[i*sample_rate:(i+1)*sample_rate] - np.mean(samples_x_B_field[i*sample_rate:(i+1)*sample_rate])
    
# samples_x_B_field = samples_x_B_field - np.mean(samples_x_B_field)
# samples_y_B_field = data_dict_y.value[0] * volts_to_nT_multiplication_factor
# samples_y_B_field = samples_y_B_field - np.mean(samples_y_B_field)


411    2.823228e+09
518    2.825370e+09
629    2.827593e+09
Name: Frequency, dtype: float64


In [6]:
result["/dev7279/demods/0/sample.x"][0].value[0]*100

array([0.00277746, 0.00277746, 0.00285347, ..., 0.00359003, 0.00352298,
       0.00352049])

# Plot Time-Traces

In [7]:
fig, axs = plt.subplots(2,1)

axs[0].plot(times[0:int(sample_rate*64)], data_dict_x.value[0][0:int(sample_rate*64)]*1e3, linewidth=0.2)
axs[0].grid()
axs[0].set_title("64 s Time Trace of Magnetic Field Noise")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("Magnetic Field [mV]")

axs[1].plot(times[0:int(sample_rate)], data_dict_x.value[0][0:int(sample_rate)]*1e3, linewidth=0.75)
axs[1].grid()
axs[1].set_title("1 s Time Trace of Magnetic Field Noise")
axs[1].set_xlabel("Time [s]")
axs[1].set_ylabel("Magnetic Field [mV]")
fig.tight_layout()
fig.show()

In [35]:
fig, axs = plt.subplots(2,1)

axs[0].plot(times[0:int(sample_rate*64)], samples_x_B_field_original[0:int(sample_rate*64)], linewidth=0.2)
axs[0].grid()
axs[0].set_title("64 s Time Trace of Magnetic Field Noise")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("Magnetic Field [nT]")

axs[1].plot(times[0:int(sample_rate)], samples_x_B_field[0:int(sample_rate)], linewidth=0.75)
axs[1].grid()
axs[1].set_title("1 s Time Trace of Magnetic Field Noise")
axs[1].set_xlabel("Time [s]")
axs[1].set_ylabel("Magnetic Field [nT]")
fig.tight_layout()
fig.show()

In [36]:
np.std(samples_x_B_field[0:int(sample_rate)])

118.40796328744098

# ASD from FFTs

In [39]:
# empty arrays for storing FFT and PSD data
# Real FFT: Computes FFT of real input signal, only half of the FFT is computed and the zero-frequency component
rffts_x = np.zeros((int(duration), int(sample_rate)//2+1))
ffts_x = np.zeros((int(duration), int(sample_rate)//2+1))
psds = np.zeros((int(duration), int(sample_rate)//2+1))

# normalization factor for the power spectral density, see "Spectrum and spectral density estimation by the Discrete Fourier transform (DFT), including a comprehensive list of window functions and some new ﬂat-top windows.""
S_1 = len(ffts_x[0])

# calculate ffts and psds for each 1 s segment of the time trace
for i in range(int(duration)):
    # rfft gives back N/2+1 complex numbers, where N is the number of samples
    rffts_x[i] = fft.rfft(samples_x_B_field[i*sample_rate:(i+1)*sample_rate])
    # fft gives back N complex numbers and we only need the first N/2+1 (positive frequencies only, including zero)
    ffts_x[i] = fft.fft(samples_x_B_field[i*sample_rate:(i+1)*sample_rate])[:int(sample_rate)//2+1]
    # Power spectral densities; # todo: Bin mir bei Faktor 2 nicht sicher, führt zu Unterschied von sqrt(2) in der Amplitude
    psds[i] = np.abs(rffts_x[i])**2 / S_1**2

psd_avg = np.mean(psds, axis=0)
asd_avg = np.sqrt(psd_avg)

fig, ax = plt.subplots()
ax.plot(asd_avg)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(np.array([1e-4, 0.15])*100)
ax.grid()

# plt.plot(times, samples_x, label='X')
fig.show()


plt.show()

C:\Users\aj92uwef\AppData\Local\Temp\ipykernel_6556\1294005866.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  rffts_x[i] = fft.rfft(samples_x_B_field[i*sample_rate:(i+1)*sample_rate])
C:\Users\aj92uwef\AppData\Local\Temp\ipykernel_6556\1294005866.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  ffts_x[i] = fft.fft(samples_x_B_field[i*sample_rate:(i+1)*sample_rate])[:int(sample_rate)//2+1]


# ASD via PSD from scipy.welch

In [40]:
fig, ax = plt.subplots()

welch_x_hanning = welch(samples_x_B_field, fs=sample_rate, nperseg=sample_rate, window='hann')
ax.plot(welch_x_hanning[0], np.sqrt(welch_x_hanning[1]), label="Hann window", alpha=0.7, linestyle="--")

welch_x_boxcar = welch(samples_x_B_field, fs=sample_rate, nperseg=sample_rate, noverlap=0, window="boxcar")
ax.plot(welch_x_boxcar[0], np.sqrt(welch_x_boxcar[1]),label="Boxcar window", alpha=0.7, linestyle="-.")

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title("Amplitude spectral density of the x-component of the magnetic field")
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Sqrt of power spectral density [nT/sqrt(Hz)]")
ax.grid()
ax.legend()
ax.set_ylim(np.array([1e-4, 0.15])*100)
fig.show()

# Compare FFT and Welch methods

In [41]:
fig, ax = plt.subplots()
axis_frequencies = welch_x_boxcar[0]
#ax.plot(axis_frequencies, np.sqrt(welch_x_hanning[1]), label="ASD from Welch, Hanning window", alpha=0.7, linestyle="-.")
ax.plot(axis_frequencies, np.sqrt(welch_x_boxcar[1]), label="ASD from Welch, Boxcar window", alpha=0.7, linestyle="-.")
ax.plot(axis_frequencies, asd_avg, label="ASD from FFT", alpha=0.7, linestyle="--")
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title("Amplitude spectral density of the x-component of the magnetic field")
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Sqrt of power spectral density [nT/sqrt(Hz)]")
ax.grid()
ax.legend()
ax.set_ylim(np.array([3e-4, 0.15])*100)
fig.show()

Die ASD von Welch und averaged periodogram method stimmen überein (macht auch Sinn, da sie ja eigentlich das exakt gleiche machen sollten, für noverlap=0).

Damit die ASD direkt aus der FFT auch mit den beiden übereinstimmt, muss ich den Faktor 2 weglassen, der laut dem "paper/manuskript" eigentlich dazugehört ... Ich glaube, das hängt irgendwie mit der single-sided FFT oder so zusammen, bin mir aber nicht ganz sicher. Ohne den Faktor 2 ists richtig. Vielleicht korrigiert scipy für die rfft den faktor 2 von alleine?

# Estimate the magnetic field sensitivity

In [42]:
f_ENBW = 500 # equivalent noise bandwidth [Hz]. Ich habe die Time Traces mit einem 500 Hz Filter 8. Ordnung gefiltert

In [43]:
np.mean([np.std(samples_x_B_field[i*int(sample_rate):(i+1)*int(sample_rate)]) for i in range(int(duration))]) / np.sqrt(2*f_ENBW)

3.6874843114656937

In [44]:
# 
np.sqrt(np.sum(welch_x_boxcar[1]) * (welch_x_boxcar[0][1] - welch_x_boxcar[0][0]) / (2*500)) 

3.6881615304875472